1. paste the link
2. from link extract information
3. from information create .md in the format like before

In [ ]:
import os
# Get the list of all files and directories
path = "D:\12-Week Project\leetcode"
dir_list = os.listdir(path)
print("Files and directories in '", path, "' :")
# prints all files
print(dir_list)